In [3]:
!pip install --quiet transformers tokenizers datasets evaluate torch

In [4]:
import transformers
print(transformers.__version__)

4.53.0


In [5]:
#Clean broken cache
!rm -rf ~/.cache/huggingface/datasets/wikitext
!rm -rf /content/hf_cache

#Downgrade fsspec to avoid glob error
!pip install -U "fsspec<2023.9.0" datasets transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 82.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.


In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", cache_dir="/content/hf_cache")

def clean_and_tokenize(example):
    if example["text"].strip() == "":
        return {"input_ids": [], "attention_mask": []}
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_attention_mask=True
    )

tokenized_datasets = dataset.map(clean_and_tokenize, batched=False, remove_columns=["text"])
tokenized_datasets = tokenized_datasets.filter(lambda x: len(x["input_ids"]) > 0)

# testing tokenizer
print(tokenized_datasets["train"][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
# grouping the data into chunks of fixed length for training and model intitialization
block_size = 128

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_datasets.map(group_texts, batched=True)


model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


Map:   0%|          | 0/2891 [00:00<?, ? examples/s]

Map:   0%|          | 0/23767 [00:00<?, ? examples/s]

Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

Embedding(50257, 768)

In [13]:
# training the model
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-wikitext2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=1,
    report_to="none",
    fp16= True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
    processing_class=tokenizer,
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.931800,1.985201
2,1.499900,1.988723
3,1.618600,2.003332


TrainOutput(global_step=35652, training_loss=1.777874472913525, metrics={'train_runtime': 3884.5331, 'train_samples_per_second': 18.355, 'train_steps_per_second': 9.178, 'total_flos': 4657595793408000.0, 'train_loss': 1.777874472913525, 'epoch': 3.0})

In [14]:
# Evaluating perplexity and top k accuracy
import math
import torch
import numpy as np
from evaluate import load

perplexity = load("perplexity")
results = perplexity.compute(model_id='gpt2', predictions=["The capital of France is"])
print("Perplexity:", results)

def compute_top_k_accuracy(logits, labels, k=5):
    _, top_k_preds = torch.topk(logits, k, dim=-1)
    correct = top_k_preds.eq(labels.unsqueeze(-1)).any(-1).float()
    return correct.mean().item()

model.eval()

# Get the device of the model
device = next(model.parameters()).device

sample = lm_dataset["validation"][0]
inputs = torch.tensor([sample["input_ids"]]).to(device) # Move inputs to the same device as the model
labels = torch.tensor([sample["labels"]]).to(device) # Move labels to the same device as the model

with torch.no_grad():
    outputs = model(inputs)
    logits = outputs.logits[:, :-1, :]
    labels = labels[:, 1:]
    top_k_acc = compute_top_k_accuracy(logits[0], labels[0])
    print(f"Top-K Accuracy: { top_k_acc*100 :.3f}% ")

  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity: {'perplexities': [59.31916809082031], 'mean_perplexity': np.float64(59.31916809082031)}
Top-K Accuracy: 97.638% 


In [24]:
trainer.save_model("./gpt2-wikitext2-final")
tokenizer.save_pretrained("./gpt2-wikitext2-final")

('./gpt2-wikitext2-final/tokenizer_config.json',
 './gpt2-wikitext2-final/special_tokens_map.json',
 './gpt2-wikitext2-final/vocab.json',
 './gpt2-wikitext2-final/merges.txt',
 './gpt2-wikitext2-final/added_tokens.json',
 './gpt2-wikitext2-final/tokenizer.json')

In [25]:
# reloading model to test
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("./gpt2-wikitext2-final")
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [42]:
from transformers import pipeline, AutoTokenizer

# Reloading model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("./gpt2-wikitext2-final")
model = GPT2LMHeadModel.from_pretrained("./gpt2-wikitext2-final")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "The theory of relativity states that"
outputs = generator(prompt, max_new_tokens=100, do_sample=True, top_k=20)

print(outputs[0]["generated_text"])

prompts = [

    "The theory of relativity states that",
    "Quantum computers are expected to",
    "Artificial intelligence can help in",
    "The capital of India is"
]
 # with temp, which allow weights to tokens with low prob
for prompt in prompts:
    print("Prompt:", prompt)
    print("GPT-2 :", generator(prompt, max_new_tokens=50, do_sample=True, top_k=40, temperature=0.8))


Device set to use cuda:0


The theory of relativity states that the speed of light travels at a speed of light. The speed of light is determined by the angle of reference between the Earth and the sun, so that the distance from the Earth to the Sun is approximately 0 @.@ 1 AU, which is 0 @.@ 5 AU from the Sun. The distance between the two stars is 0 @.@ 2 AU, which is 0 @.@ 3 AU from the Sun. The distance between the two stars is approximately 0 @.@ 8 AU
Prompt: The theory of relativity states that
GPT-2 : [{'generated_text': "The theory of relativity states that a given particle or cell of matter is always in a state of equilibrium with its surrounding environment, and that the particles around it are always in equilibrium with their environment. However, a particle's equilibrium with its environment is not known until it encounters a"}]
Prompt: Quantum computers are expected to
GPT-2 : [{'generated_text': 'Quantum computers are expected to replace traditional typewriters in the near future, and some researcher